In [1]:
from datetime import datetime
from datetime import timedelta
import json
import re
from os import path, environ
from time import sleep
import os

import requests
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver      # imports
from selenium.webdriver.support.ui import Select

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
base_url = 'https://www.gunviolencearchive.org'

In [3]:
#These files need to be in the same directory as the jupyter notebook file.
SHOOTINGS_JSON = 'shootings.json'

SHOOTINGS_COMPACT_JSON = 'shootings_compact.json'

### Function declarations:

In [4]:

def scrape_results(current_url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
    url = current_url
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, features="lxml")
    table = soup.find("table")
    headers = [
        re.sub(r"[^a-z]", "", th.get_text().lower()) for th in table.find_all("th")
    ]
    rows = table.find("tbody").find_all("tr")

    results = []
    for row in rows:
        cols = row.find_all("td")
        links = row.select(".links a")
        incident_id = int(re.sub(r"[^\d]", "", links[0].get("href")))
        if not incident_id > 0:
            raise RuntimeError(f"Invalid incident ID {incident_id} at {url}")
        results.append(
            {
                "id": int(re.sub(r"[^\d]", "", links[0].get("href"))),
                "date": parse(cols[headers.index("incidentdate")].get_text()),
                "state": cols[headers.index("state")].get_text(),
                "city": cols[headers.index("cityorcounty")].get_text(),
                "address": cols[headers.index("address")].get_text(),
                "killed": int(cols[headers.index("killed")].get_text()),
                "injured": int(cols[headers.index("injured")].get_text()),
                "source": links[1].get("href") if len(links) > 1 else None,
            }
        )
        
    return results

In [5]:
def merge_shooting_data(scraped_data):
    total_new = 0
    for shooting in scraped_data:
        if shooting["id"] not in keyed_shooting_data:
            print(f'New shooting found: {shooting["id"]}, {shooting["date"]}')
            keyed_shooting_data[shooting["id"]] = {}
            total_new += 1
        keyed_shooting_data[shooting["id"]].update(shooting)
    return total_new

In [14]:

def parse_page(next_url):
    # HTTP GET requests
    page = requests.get(next_url)

    # Checking if we successfully fetched the URL
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'lxml')
        try:
            current_url = next_url
            print(current_url)
            results = scrape_results(current_url)
            merge_shooting_data(results)
            next_page_text = bs.find('ul', class_="pager").findAll('li')[-2].text
            if next_page_text == 'next ›':
                next_page_partial = bs.find('ul', class_="pager").findAll('li')[-2].find('a')['href']
                next_page_url = base_url + next_page_partial
                current_url = next_page_url
                print(next_page_url)
                results = scrape_results(current_url)
                merge_shooting_data(results)
                sleep(3)
                if next_page_text == 'next ›':
                    parse_page(next_page_url)
                    sleep(1)
                else:
                    merge_shooting_data(results)
        except:
            current_url = next_url
            results = scrape_results(current_url)
            merge_shooting_data(results)
            

### Main implementation:

In [21]:
# Change the year, and months based on the date range you need to scarape data for. 
# So if you want data from 12th-April-2020 to 22nd-April-2020, remove all the years prior to 2020;
# and in month remove all months except 'Apr'.
# To get an even more specific date range (to save time) you can remove the corresponding days of that month from day[] in
# the corresponding if statement for that month. 
year = ['2014','2015','2016','2017','2018','2019','2020']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            if month[months]=='Mar':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Apr':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29] 
            if month[months]=='May':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Jun':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Jul':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Aug':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Sep':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Oct':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Nov':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            
        else:
            if month[months]=='Jan':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
            if month[months]=='Mar':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Apr':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29] 
            if month[months]=='May':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Jun':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Jul':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Aug':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Sep':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Oct':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Nov':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
            
                driver = webdriver.Chrome('C:\Webdrivers\chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')



                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)


                # Choose Date

                driver.find_element_by_link_text('Date').click()

                # 4 seconds for page loading
                sleep(4)

                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(4)
                month1.select_by_visible_text(month[months])
                sleep(4)



                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year1.select_by_visible_text(str(year[years]))

                sleep(3)

                # start date

                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()

                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()

                # month select
                sleep(3)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(3)
                month2.select_by_visible_text(month[months])
                sleep(3)


                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year2.select_by_visible_text(str(year[years]))

                sleep(3)

                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()

                sleep(3)
                # Click Search

                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                parse_page(current_url)
                print("Sorting results")
                updated_shooting_data = sorted(
                    keyed_shooting_data.values(), key=lambda row: row['date']
                )

                updated_shootings_compact = []
                for shooting in updated_shooting_data:
                    shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                    updated_shootings_compact.append(
                        [
                            shooting["id"],
                            shooting["date"],
                            shooting["killed"],
                            shooting["injured"],

                        ]
                    )


                print("Writing results")
                with open(SHOOTINGS_JSON, "w") as outfile:
                    json.dump(updated_shooting_data, outfile, indent=2)

                with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                    json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

                print(f"Done! {len(updated_shooting_data)} shootings found")
                driver.close()
            else:
                break
            


Loading existing shooting data with 65137 incidents
https://www.gunviolencearchive.org/query/082449f1-31ef-4caf-bdd3-7ba8830eff11
New shooting found: 1586171, 2020-01-01 00:00:00
New shooting found: 1586139, 2020-01-01 00:00:00
New shooting found: 1585484, 2020-01-01 00:00:00
New shooting found: 1585746, 2020-01-01 00:00:00
New shooting found: 1586051, 2020-01-01 00:00:00
New shooting found: 1586146, 2020-01-01 00:00:00
New shooting found: 1587164, 2020-01-01 00:00:00
New shooting found: 1586675, 2020-01-01 00:00:00
New shooting found: 1587149, 2020-01-01 00:00:00
New shooting found: 1587134, 2020-01-01 00:00:00
New shooting found: 1586031, 2020-01-01 00:00:00
New shooting found: 1591514, 2020-01-01 00:00:00
New shooting found: 1586122, 2020-01-01 00:00:00
New shooting found: 1589234, 2020-01-01 00:00:00
New shooting found: 1586119, 2020-01-01 00:00:00
New shooting found: 1620983, 2020-01-01 00:00:00
New shooting found: 1586114, 2020-01-01 00:00:00
New shooting found: 1587110, 2020-01-

Writing results
Done! 65287 shootings found
Loading existing shooting data with 65287 incidents
https://www.gunviolencearchive.org/query/a63ee43f-99d2-4a0b-899c-1217ee1a2480
New shooting found: 1591093, 2020-01-07 00:00:00
New shooting found: 1591361, 2020-01-07 00:00:00
New shooting found: 1591047, 2020-01-07 00:00:00
New shooting found: 1591184, 2020-01-07 00:00:00
New shooting found: 1591033, 2020-01-07 00:00:00
New shooting found: 1591031, 2020-01-07 00:00:00
New shooting found: 1592642, 2020-01-07 00:00:00
New shooting found: 1591849, 2020-01-07 00:00:00
New shooting found: 1590982, 2020-01-07 00:00:00
New shooting found: 1591847, 2020-01-07 00:00:00
New shooting found: 1591841, 2020-01-07 00:00:00
New shooting found: 1592565, 2020-01-07 00:00:00
New shooting found: 1590685, 2020-01-07 00:00:00
New shooting found: 1592557, 2020-01-07 00:00:00
New shooting found: 1594538, 2020-01-07 00:00:00
New shooting found: 1597436, 2020-01-07 00:00:00
New shooting found: 1590994, 2020-01-07 00

Writing results
Done! 65437 shootings found
Loading existing shooting data with 65437 incidents
https://www.gunviolencearchive.org/query/f923064f-5809-4a3e-a6e2-27b4c3e1f907
New shooting found: 1595581, 2020-01-13 00:00:00
New shooting found: 1595577, 2020-01-13 00:00:00
New shooting found: 1595574, 2020-01-13 00:00:00
New shooting found: 1595572, 2020-01-13 00:00:00
New shooting found: 1595642, 2020-01-13 00:00:00
New shooting found: 1595699, 2020-01-13 00:00:00
New shooting found: 1595549, 2020-01-13 00:00:00
New shooting found: 1596334, 2020-01-13 00:00:00
New shooting found: 1596585, 2020-01-13 00:00:00
New shooting found: 1595534, 2020-01-13 00:00:00
New shooting found: 1596579, 2020-01-13 00:00:00
New shooting found: 1596571, 2020-01-13 00:00:00
New shooting found: 1595517, 2020-01-13 00:00:00
New shooting found: 1596541, 2020-01-13 00:00:00
New shooting found: 1596532, 2020-01-13 00:00:00
New shooting found: 1596527, 2020-01-13 00:00:00
New shooting found: 1596492, 2020-01-13 00

Writing results
Done! 65587 shootings found
Loading existing shooting data with 65587 incidents
https://www.gunviolencearchive.org/query/6078dae4-23f0-4bbc-9d5e-e5ffcbbab938
New shooting found: 1601722, 2020-01-19 00:00:00
New shooting found: 1601719, 2020-01-19 00:00:00
New shooting found: 1601032, 2020-01-19 00:00:00
New shooting found: 1601711, 2020-01-19 00:00:00
New shooting found: 1602371, 2020-01-19 00:00:00
New shooting found: 1622800, 2020-01-19 00:00:00
New shooting found: 1601600, 2020-01-19 00:00:00
New shooting found: 1601685, 2020-01-19 00:00:00
New shooting found: 1603366, 2020-01-19 00:00:00
New shooting found: 1601677, 2020-01-19 00:00:00
New shooting found: 1604588, 2020-01-19 00:00:00
New shooting found: 1600530, 2020-01-19 00:00:00
New shooting found: 1602491, 2020-01-19 00:00:00
New shooting found: 1601668, 2020-01-19 00:00:00
New shooting found: 1600516, 2020-01-19 00:00:00
New shooting found: 1601663, 2020-01-19 00:00:00
New shooting found: 1602463, 2020-01-19 00

Writing results
Done! 65737 shootings found
Loading existing shooting data with 65737 incidents
https://www.gunviolencearchive.org/query/0ee9be18-ad6d-4986-a77b-652c5c0b8578
New shooting found: 1606285, 2020-01-25 00:00:00
New shooting found: 1605327, 2020-01-25 00:00:00
New shooting found: 1605296, 2020-01-25 00:00:00
New shooting found: 1605469, 2020-01-25 00:00:00
New shooting found: 1605702, 2020-01-25 00:00:00
New shooting found: 1605460, 2020-01-25 00:00:00
New shooting found: 1605454, 2020-01-25 00:00:00
New shooting found: 1605452, 2020-01-25 00:00:00
New shooting found: 1605571, 2020-01-25 00:00:00
New shooting found: 1605424, 2020-01-25 00:00:00
New shooting found: 1606027, 2020-01-25 00:00:00
New shooting found: 1605335, 2020-01-25 00:00:00
New shooting found: 1607177, 2020-01-25 00:00:00
New shooting found: 1605384, 2020-01-25 00:00:00
New shooting found: 1607179, 2020-01-25 00:00:00
New shooting found: 1605609, 2020-01-25 00:00:00
New shooting found: 1605226, 2020-01-25 00

...

Writing results
Done! 67087 shootings found
Loading existing shooting data with 67087 incidents
https://www.gunviolencearchive.org/query/de175755-fd4a-442b-b9fc-421773e43f8b
New shooting found: 1642239, 2020-03-19 00:00:00
New shooting found: 1643123, 2020-03-19 00:00:00
New shooting found: 1643117, 2020-03-19 00:00:00
New shooting found: 1642368, 2020-03-19 00:00:00
New shooting found: 1642493, 2020-03-19 00:00:00
New shooting found: 1643110, 2020-03-19 00:00:00
New shooting found: 1644060, 2020-03-19 00:00:00
New shooting found: 1642487, 2020-03-19 00:00:00
New shooting found: 1643425, 2020-03-19 00:00:00
New shooting found: 1642477, 2020-03-19 00:00:00
New shooting found: 1643104, 2020-03-19 00:00:00
New shooting found: 1642444, 2020-03-19 00:00:00
New shooting found: 1642442, 2020-03-19 00:00:00
New shooting found: 1642440, 2020-03-19 00:00:00
New shooting found: 1643210, 2020-03-19 00:00:00
New shooting found: 1644002, 2020-03-19 00:00:00
New shooting found: 1642428, 2020-03-19 00

Writing results
Done! 67237 shootings found
Loading existing shooting data with 67237 incidents
https://www.gunviolencearchive.org/query/5dd70517-5bd0-4897-91ef-132293ef2001
New shooting found: 1645781, 2020-03-25 00:00:00
New shooting found: 1645845, 2020-03-25 00:00:00
New shooting found: 1650992, 2020-03-25 00:00:00
New shooting found: 1645685, 2020-03-25 00:00:00
New shooting found: 1646342, 2020-03-25 00:00:00
New shooting found: 1646339, 2020-03-25 00:00:00
New shooting found: 1645768, 2020-03-25 00:00:00
New shooting found: 1645757, 2020-03-25 00:00:00
New shooting found: 1646316, 2020-03-25 00:00:00
New shooting found: 1645875, 2020-03-25 00:00:00
New shooting found: 1645746, 2020-03-25 00:00:00
New shooting found: 1651705, 2020-03-25 00:00:00
New shooting found: 1645735, 2020-03-25 00:00:00
New shooting found: 1647004, 2020-03-25 00:00:00
New shooting found: 1646299, 2020-03-25 00:00:00
New shooting found: 1646018, 2020-03-25 00:00:00
New shooting found: 1647382, 2020-03-25 00

Writing results
Done! 67387 shootings found
Loading existing shooting data with 67387 incidents
https://www.gunviolencearchive.org/query/feefd793-3a16-4226-a3c3-43436122454e
New shooting found: 1650104, 2020-03-31 00:00:00
New shooting found: 1649452, 2020-03-31 00:00:00
New shooting found: 1651120, 2020-03-31 00:00:00
New shooting found: 1650160, 2020-03-31 00:00:00
New shooting found: 1649203, 2020-03-31 00:00:00
New shooting found: 1650987, 2020-03-31 00:00:00
New shooting found: 1650150, 2020-03-31 00:00:00
New shooting found: 1650148, 2020-03-31 00:00:00
New shooting found: 1656568, 2020-03-31 00:00:00
New shooting found: 1650145, 2020-03-31 00:00:00
New shooting found: 1651735, 2020-03-31 00:00:00
New shooting found: 1649813, 2020-03-31 00:00:00
New shooting found: 1649426, 2020-03-31 00:00:00
New shooting found: 1649549, 2020-03-31 00:00:00
New shooting found: 1650122, 2020-03-31 00:00:00
New shooting found: 1649600, 2020-03-31 00:00:00
New shooting found: 1649471, 2020-03-31 00

Writing results
Done! 67537 shootings found
Loading existing shooting data with 67537 incidents
https://www.gunviolencearchive.org/query/6a500e77-fd05-4194-bcce-eeeaa0b92f74
New shooting found: 1653424, 2020-04-06 00:00:00
New shooting found: 1653074, 2020-04-06 00:00:00
New shooting found: 1654271, 2020-04-06 00:00:00
New shooting found: 1655170, 2020-04-06 00:00:00
New shooting found: 1653473, 2020-04-06 00:00:00
New shooting found: 1653461, 2020-04-06 00:00:00
New shooting found: 1653286, 2020-04-06 00:00:00
New shooting found: 1653707, 2020-04-06 00:00:00
New shooting found: 1656554, 2020-04-06 00:00:00
New shooting found: 1654262, 2020-04-06 00:00:00
New shooting found: 1654032, 2020-04-06 00:00:00
New shooting found: 1653430, 2020-04-06 00:00:00
New shooting found: 1655097, 2020-04-06 00:00:00
New shooting found: 1653429, 2020-04-06 00:00:00
New shooting found: 1653428, 2020-04-06 00:00:00
New shooting found: 1653255, 2020-04-06 00:00:00
New shooting found: 1653417, 2020-04-06 00

Writing results
Done! 67687 shootings found
Loading existing shooting data with 67687 incidents
https://www.gunviolencearchive.org/query/2bd2c860-8b4d-44e1-a0e4-ab7bb980b6f3
New shooting found: 1657688, 2020-04-12 00:00:00
New shooting found: 1658353, 2020-04-12 00:00:00
New shooting found: 1658340, 2020-04-12 00:00:00
New shooting found: 1658332, 2020-04-12 00:00:00
New shooting found: 1657735, 2020-04-12 00:00:00
New shooting found: 1658338, 2020-04-12 00:00:00
New shooting found: 1657733, 2020-04-12 00:00:00
New shooting found: 1657731, 2020-04-12 00:00:00
New shooting found: 1657726, 2020-04-12 00:00:00
New shooting found: 1658038, 2020-04-12 00:00:00
New shooting found: 1657723, 2020-04-12 00:00:00
New shooting found: 1658031, 2020-04-12 00:00:00
New shooting found: 1657721, 2020-04-12 00:00:00
New shooting found: 1658304, 2020-04-12 00:00:00
New shooting found: 1658282, 2020-04-12 00:00:00
New shooting found: 1657709, 2020-04-12 00:00:00
New shooting found: 1658296, 2020-04-12 00

## Test code 

In [29]:
# # TEST LOOP:

# year = ["2019","2020"]
# month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# count=0
# for years in range(len(year)): 
    
#     for months in range(len(month)):
        
                
#         print(day)
        
           
            



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2

In [ ]:
# Opening and reading files test code:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

In [15]:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

# # Insert any GVA date query url link into the parse page function to test the code
# parse_page('https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f')
# print("Sorting results")
# updated_shooting_data = sorted(
#     keyed_shooting_data.values(), key=lambda row: row['date']
# )

# updated_shootings_compact = []
# for shooting in updated_shooting_data:
#     shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#     updated_shootings_compact.append(
#         [
#             shooting["id"],
#             shooting["date"],
#             shooting["killed"],
#             shooting["injured"],

#         ]
#     )


# print("Writing results")
# with open(SHOOTINGS_JSON, "w") as outfile:
#     json.dump(updated_shooting_data, outfile, indent=2)

# with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
#     json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

# print(f"Done! {len(updated_shooting_data)} shootings found")

Loading existing shooting data with 56112 incidents
https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f
Sorting results
Writing results
Done! 56112 shootings found


In [38]:
# y = ['2020']
# m = ['Apr']
# d = [10]
# for ys in range(len(y)):
#     for ms in range(len(m)):
#         for ds in range(len(d)):
#             c_date = '{}-{}-{}'.format(y[ys],m[ms],d[ds])
#             c_date = datetime.strptime(c_date,"%Y-%b-%d")
#             day_difference = (datetime.now() - c_date).days
#             print(day_difference)
#             if (day_difference >3):
#                 print(c_date)
#             else:
#                 print("cats")

#### Geocoder test code

In [ ]:
# for year in range(start_year, current_year + 1):
#     print(f"scraping year {year}")
#     page = 0
#     done_importing = False
#     while not done_importing:
#         results = scrape_results(year, page, headers)
#         new_results_count = merge_shooting_data(results)
#         done_importing = new_results_count == 0
#         page += 1
#         sleep(1)
#     print(f"Done importing {year}")

# # print("Geocoding missing data")
# # for shooting in keyed_mass_shooting_data.values():
# #     if not shooting.get("lat") or not shooting.get("lng"):
# #         loc = geocode(shooting["address"], shooting["city"], shooting["state"])
# #         shooting["lat"] = loc.latitude
# #         shooting["lng"] = loc.longitude
# #         print(f"geocoded {loc.address}: {loc.latitude}, {loc.longitude}")

#### Update results method test (did not use in implementation)

In [ ]:
# def update_results():

#     start_year = 2014
#     current_year = datetime.now().year
#     with open(ACCIDENTAL_SHOOTINGS_JSON) as json_file:
#         accidental_shooting_data = json.load(json_file)

#     keyed_accidental_shooting_data = {}
#     for shooting in accidental_shooting_data:
#         keyed_accidental_shooting_data[shooting["id"]] = shooting
#         shooting["date"] = parse(shooting["date"])

#     print(f"Loading existing accidental shooting data with {len(accidental_shooting_data)} incidents")
#     print("Sorting results")
#     updated_shooting_data = sorted(
#         keyed_accidental_shooting_data.values(), key=lambda row: row['date']
#     )

#     updated_shootings_compact = []
#     for shooting in updated_shooting_data:
#         shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#         updated_shootings_compact.append(
#             [
#                 shooting["id"],
#                 shooting["date"],
#                 shooting["killed"],
#                 shooting["injured"],

#             ]
#         )


#     print("Writing results")
#     with open(ACCIDENTAL_SHOOTINGS_JSON, "w") as outfile:
#         json.dump(updated_shooting_data, outfile, indent=2)

#     with open(ACCIDENTAL_SHOOTINGS_COMPACT_JSON, "w") as outfile:
#         json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

#     print(f"Done! {len(updated_shooting_data)} shootings found")